In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from google.colab import drive
drive.mount('/content/drive')
from glob import glob
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary
from fastai.metrics import accuracy
from fastai.vision import *
import numpy as np
!pip install fft-conv-pytorch
from fft_conv_pytorch import fft_conv, FFTConv2d

Mounted at /content/drive


In [ ]:
#Run only 1st time to download dataset into drive. Dont run if dataset is already downloaded to drive
# ! wget -P /content/drive/MyDrive https://github.com/muhammedtalo/COVID-19/archive/refs/heads/master.zip
# ! unzip -q /content/drive/MyDrive/master.zip -d /content/drive/MyDrive/Dataset

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# path = "/content/drive/MyDrive/covid-19_dataset_grayscale"
# np.random.seed(41)
# data = ImageDataBunch.from_folder(path, train="Train", valid ="Valid", valid_pct=0.2,ds_tfms=get_transforms(), size=(1,512,512), bs=16, num_workers=4).normalize()

In [4]:
from fastai.vision.data import normalize
path = "/content/drive/MyDrive/Dataset/brain_tumor_dataset_grayscale"
np.random.seed(41)
defaults = SimpleNamespace(cmap='L', return_fig=False, silent=False)
data = ImageDataBunch.from_folder(path, train="Train", valid ="Valid", valid_pct=0.2,ds_tfms=get_transforms(), size=(256,256), bs=16, num_workers=4)
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"

In [5]:
print (data.classes, data.c, len(data.train_ds), len(data.valid_ds))
print (data.train_ds.classes)
print (data.valid_ds.classes)

['no', 'yes'] 2 203 50
['no', 'yes']
['no', 'yes']


With FFT Conv Layers

In [ ]:
# def fft_conv_block(ni, nf, size=3, stride=1):
#     for_pad = lambda s: s if s > 2 else 3
#     return nn.Sequential(
#         FFTConv2d(ni, nf, kernel_size=size, stride=stride,
#                   padding=(for_pad(size) - 1)//2, bias=False), 
#         nn.BatchNorm2d(nf),
#         nn.LeakyReLU(negative_slope=0.1, inplace=True)  
#     )

# def conv_block(ni, nf, size=3, stride=1):
#     for_pad = lambda s: s if s > 2 else 3
#     return nn.Sequential(
#         nn.Conv2d(ni, nf, kernel_size=size, stride=stride,
#                   padding=(for_pad(size) - 1)//2, bias=False), 
#         nn.BatchNorm2d(nf),
#         nn.LeakyReLU(negative_slope=0.1, inplace=True)  
#     )

# def triple_conv(ni, nf, size=3):
#     return nn.Sequential(
#         conv_block(ni, nf,size),
#         conv_block(nf, ni, size=1),  
#         conv_block(ni, nf,size)
#     )
# def fft_triple_conv(ni, nf,size):
#     return nn.Sequential(
#         fft_conv_block(ni, nf,size),
#         conv_block(nf, ni, size=1),  
#         fft_conv_block(ni, nf, size)
#     )
# 21 17

In [28]:


vgg_model = nn.Sequential(
    
    nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2), 

    nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),

    nn.MaxPool2d(kernel_size=2, stride=2), 

    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2), 
    
    # nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.MaxPool2d(kernel_size=2, stride=2), 
    
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    # nn.ReLU(inplace=True),
    # nn.MaxPool2d(kernel_size=2, stride=2), 

    nn.Flatten(),
    nn.Linear(65536 , 512),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    # nn.Linear(4096, 4096),
    # nn.ReLU(inplace=True),
    # nn.Dropout(0.5),
    nn.Linear(512, 2)
)





In [29]:
vgg_learn = Learner(data, vgg_model, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)
print(vgg_learn.summary())

Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [8, 256, 256]        80         True      
______________________________________________________________________
ReLU                 [8, 256, 256]        0          False     
______________________________________________________________________
Conv2d               [8, 256, 256]        584        True      
______________________________________________________________________
ReLU                 [8, 256, 256]        0          False     
______________________________________________________________________
MaxPool2d            [8, 128, 128]        0          False     
______________________________________________________________________
Conv2d               [16, 128, 128]       1,168      True      
______________________________________________________________________
ReLU                 [16, 128, 128]       0          False     
___________________________________________________

In [30]:
import time
start = time.time()
vgg_learn.fit_one_cycle(100, max_lr=8e-5)
end = time.time()
print(end-start)

epoch,train_loss,valid_loss,accuracy,time
0,0.695053,0.688158,0.620000,00:03
1,0.689269,0.680474,0.620000,00:03
2,0.683735,0.673025,0.620000,00:03
3,0.680072,0.667319,0.620000,00:03
4,0.677172,0.664519,0.620000,00:03
5,0.675143,0.664063,0.620000,00:03
6,0.674278,0.664046,0.620000,00:03
7,0.673713,0.664051,0.620000,00:03
8,0.672639,0.664779,0.620000,00:03
9,0.672461,0.664258,0.620000,00:03


382.8888819217682
